In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_groq import ChatGroq

from langgraph.graph.message import add_messages


In [ ]:
class ChatState(TypedDict):
    messages : Annotated[list[BaseMessage], add_messages] #we will keep adding messages here

In [ ]:
import os
# api_key=os.getenv("GROQ_API_KEY")
api_key=''
llm = ChatGroq(model='llama-3.1-8b-instant',api_key=api_key)

In [4]:
def chat_node(state : ChatState)->ChatState:
    messages = state['messages']
    response = llm.invoke(messages)

    return {'messages' :[response]}

In [5]:
graph = StateGraph(ChatState)

graph.add_node('chat_node',chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node',END)

chatbot = graph.compile()


In [6]:
initial_state = {
    'messages' :[HumanMessage(content='What is the capital of India')]
}

In [7]:
chatbot.invoke(initial_state)['messages'][-1].content

'The capital of India is New Delhi.'

In [8]:
while True:
    user_message = input('Type here: ')

    print('user : ',user_message)

    if user_message.strip().lower() in ['exit','quit','bye']:
        break

    response = chatbot.invoke({'messages' : [HumanMessage(content = user_message)]})

    print('AI' , response['messages'][-1].content)

user :  exit


In [9]:
from langgraph.checkpoint.memory import MemorySaver

In [13]:
checkpointer =MemorySaver()

graph = StateGraph(ChatState)

graph.add_node('chat_node' , chat_node)

graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

chatbot = graph.compile(checkpointer=checkpointer)

In [14]:
thread_id ='1'

while True:
    user_message = input('Type here: ')
    print('User message ',user_message)

    if user_message.strip().lower() in ['exit','quit','bye']:
        break
    config = {'configurable' : {'thread_id' : thread_id}}

    response = chatbot.invoke({'messages' : [HumanMessage(content=user_message)]} , config=config)

    print('AI : ',response['messages'][-1].content)

User message  hey ,my name is priya
AI :  Nice to meet you, Priya. Is there something I can help you with or would you like to chat?
User message  can you plz tell my name
AI :  Your name is Priya.
User message  how did you know
AI :  You told me earlier, when you introduced yourself by saying "hey, my name is Priya".
User message  so, will you save all info that i give you
AI :  For our conversation, I have a limited ability to remember context and previous messages. However, I don't retain any information about your identity or personal data once our conversation is closed.

Each time you interact with me, it's a new conversation, and I start from a clean slate. I use natural language processing to understand and respond to your input in real-time, but I don't store any data about you or our conversation.

If you want to share sensitive information or have a follow-up conversation, feel free to share it again, and I'll do my best to help!
User message  exit


In [15]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='hey ,my name is priya', additional_kwargs={}, response_metadata={}, id='e394fd37-1a29-4601-8f88-c51b6ec14146'), AIMessage(content='Nice to meet you, Priya. Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 42, 'total_tokens': 66, 'completion_time': 0.037516665, 'completion_tokens_details': None, 'prompt_time': 0.002454867, 'prompt_tokens_details': None, 'queue_time': 0.055624118, 'total_time': 0.039971532}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb5c4-0b7c-70f1-97ed-308f8af9a77e-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 42, 'output_tokens': 24, 'total_tokens': 66}), HumanMessage(content='can you plz tell my name', additional_kwargs={}

In [ ]:
#if config was different ,it will be in different rows